In [4]:
from pyspark.sql.types import *
import pyspark.sql.functions as fn
from delta import *
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar,
                                 /home/jovyan/jars/postgresql-42.3.3.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "ZiVYnHFvBt2HumCAXmQG")
         .config("spark.hadoop.fs.s3a.secret.key", "oVNgBEZ5PISLiwF25fJkoazHGXRFB7mJeiBcVygd")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

df_delta = spark.read.format("delta").load('s3a://silver/prd_yellow_taxi_table')

df_delta.show()


+--------+---------------+------------+--------------------+---------------------+
|VendorID|passenger_count|total_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------+---------------+------------+--------------------+---------------------+
|       1|              2|         9.4| 2023-02-01 00:32:53|  2023-02-01 00:34:34|
|       2|              1|        -5.5| 2023-02-01 00:35:16|  2023-02-01 00:35:30|
|       2|              1|         5.5| 2023-02-01 00:35:16|  2023-02-01 00:35:30|
|       1|              0|       74.65| 2023-02-01 00:29:33|  2023-02-01 01:01:38|
|       2|              1|        25.3| 2023-02-01 00:12:28|  2023-02-01 00:25:46|
|       1|              1|       32.25| 2023-02-01 00:52:40|  2023-02-01 01:07:18|
|       1|              1|        50.0| 2023-02-01 00:12:39|  2023-02-01 00:40:36|
|       1|              1|       14.64| 2023-02-01 00:56:53|  2023-02-01 01:00:37|
|       2|              1|       44.12| 2023-02-01 00:20:40|  2023-02-01 00:33:56|
|   

### 1. Média de total_amount recebido em um mês

In [8]:
from pyspark.sql.functions import col, avg, date_format, hour, dayofmonth, round

# Extraindo o mês
df_monthly = df_delta.withColumn("month", date_format(col("tpep_pickup_datetime"), "yyyy-MM"))

# Calculando a média do valor total por mês
avg_total_amount_monthly = (
    df_monthly.groupBy("month")
    .agg(round(avg("total_amount"),2).alias("avg_total_amount"))
)

avg_total_amount_monthly.show()


+-------+----------------+
|  month|avg_total_amount|
+-------+----------------+
|2023-03|            27.8|
|2023-02|            26.9|
|2023-01|           27.02|
|2023-04|           28.27|
|2023-05|           28.96|
+-------+----------------+



### 2. Média de passenger_count por hora e por dia

In [12]:
# Extraindo hora e dia
df_hourly_daily = df_delta.withColumn("hour", hour(col("tpep_pickup_datetime"))).withColumn(
    "day", dayofmonth(col("tpep_pickup_datetime"))
)

# Calculando a média de passageiros por hora e por dia
avg_passengers_hourly_daily = (
    df_hourly_daily.groupBy("day", "hour")
    .agg(avg("passenger_count").alias("avg_passenger_count"))
    .orderBy('day', 'hour')
)

avg_passengers_hourly_daily.show(200, truncate=False)


+---+----+-------------------+
|day|hour|avg_passenger_count|
+---+----+-------------------+
|1  |0   |1.4539021034008257 |
|1  |1   |1.4794662831575582 |
|1  |2   |1.4919833077092026 |
|1  |3   |1.4407545993140005 |
|1  |4   |1.4171229467396715 |
|1  |5   |1.304796599878567  |
|1  |6   |1.2404274265360642 |
|1  |7   |1.2722914669223393 |
|1  |8   |1.2528586137755378 |
|1  |9   |1.2727228588075354 |
|1  |10  |1.330741159092884  |
|1  |11  |1.341086858593718  |
|1  |12  |1.3622344960521926 |
|1  |13  |1.3736038048569577 |
|1  |14  |1.374952324815367  |
|1  |15  |1.3924929413718652 |
|1  |16  |1.3853626517886446 |
|1  |17  |1.3804608518777133 |
|1  |18  |1.3578868403589255 |
|1  |19  |1.3851577924723433 |
|1  |20  |1.3940294429074016 |
|1  |21  |1.3921234778102676 |
|1  |22  |1.3975889041206355 |
|1  |23  |1.4150410543381622 |
|2  |0   |1.4004221807258261 |
|2  |1   |1.4312622309197651 |
|2  |2   |1.4356955380577427 |
|2  |3   |1.450295691354548  |
|2  |4   |1.380518889394629  |
|2  |5  